In [1]:
import time
import argparse
import math
import random
import numpy as np

import cv2
import itertools
import os
from os import listdir
from os.path import isfile, join


from sklearn.svm import SVC
from operator import sub

import pickle #save and read parameters with SVM 

np.set_printoptions(precision=2)

import openface

from scipy.spatial import Delaunay

'''#cross-validation
clf = []
for f in xrange(5):
    clf.append(SVC())
'''
#train and test
clf = SVC(C=1e3) 


#fileDir = os.path.dirname(os.path.realpath(__file__))
modelDir = os.path.join( '..', 'Models')
dlibModelDir = os.path.join(modelDir, 'dlib')
openfaceModelDir = os.path.join(modelDir, 'openface')

dlibFacePredictor = os.path.join(dlibModelDir, "shape_predictor_68_face_landmarks.dat")
networkModel = os.path.join(openfaceModelDir, 'nn4.small2.v1.t7')
imgDim = 96

start = time.time()
align = openface.AlignDlib(dlibFacePredictor)
net = openface.TorchNeuralNet(networkModel, imgDim)

triangular_list = []

mypath = '../Data'
plastic = [f for f in listdir(mypath+'/plastic') if f.lower().endswith(('.png', '.jpg', '.jpeg')) and 
                                                    isfile(join(mypath+'/plastic', f))]
american = [f for f in listdir(mypath+'/american') if f.lower().endswith(('.png', '.jpg', '.jpeg')) and 
                                                      isfile(join(mypath+'/american', f))]
korean = [f for f in listdir(mypath+'/korean') if f.lower().endswith(('.png', '.jpg', '.jpeg')) and 
                                                  isfile(join(mypath+'/korean', f))]


from BeautificationEngine import BE

vector = []
label = []

be = BE(align, net, mypath, imgDim, american, korean, plastic)
be.get_triangle_list_from_data('../Models/triIdx.txt')
be.getedge(68)
#train whole data and test whole data
start = 49
end = 0
vector, label = be.get_image_data(start,end)

print(len(vector))

clf = be.svm(clf, vector, label)
be.saveModel(clf)
print('SVM Training is done!')

150
SVM Training is done!


In [6]:
start = 0
end = 49
clf2 = be.loadModel()
t,total = be.test(clf2,start,end)
print('Accuracy: {}'.format(1.0*t/total))

Accuracy: 0.897959183673
